<a href="https://colab.research.google.com/github/bobfromthemailroom/Prompt-Builder/blob/main/Prompt_Biilder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install datasets pandas ipywidgets

import ipywidgets as widgets
from datasets import load_dataset
import pandas as pd
from IPython.display import display, HTML, clear_output
import json
from datetime import datetime

class PromptBuilder:
    def __init__(self):
        self.df = self.load_prompt_dataset()
        self.create_widgets()
        self.setup_layout()

    def load_prompt_dataset(self):
        dataset = load_dataset("fka/awesome-chatgpt-prompts")
        return pd.DataFrame(dataset['train'])

    def create_widgets(self):
        # Search widget for examples
        self.search_input = widgets.Text(
            placeholder='Search examples...',
            description='',  # Removed label
            layout=widgets.Layout(width='100%')
        )

        # Example list with search functionality
        self.example_list = widgets.Select(
            options=[('Select an example...', -1)] + [(f"{row['act']}", i) for i, row in self.df.iterrows()],
            value=-1,
            rows=5,
            description='',  # Removed label
            layout=widgets.Layout(width='100%')
        )

        # Main input fields
        self.task_input = widgets.Textarea(
            placeholder='What do you want the AI to do? Be specific and clear.',
            description='',  # Removed label
            layout=widgets.Layout(width='100%', height='100px')
        )

        self.context_input = widgets.Textarea(
            placeholder='Provide relevant background information or context',
            description='',  # Removed label
            layout=widgets.Layout(width='100%', height='100px')
        )

        self.constraints_input = widgets.Textarea(
            placeholder='List any limitations or requirements',
            description='',  # Removed label
            layout=widgets.Layout(width='100%', height='100px')
        )

        self.format_input = widgets.Textarea(
            placeholder='Specify how you want the response structured',
            description='',  # Removed label
            layout=widgets.Layout(width='100%', height='100px')
        )

        # Tone selection with custom option
        self.tone_dropdown = widgets.Dropdown(
            options=['Professional', 'Casual', 'Academic', 'Creative', 'Technical', 'Custom'],
            value='Professional',
            description='',  # Removed label
            layout=widgets.Layout(width='100%')
        )

        self.custom_tone_input = widgets.Text(
            placeholder='Enter custom tone',
            description='',  # Removed label
            layout=widgets.Layout(width='100%'),
            disabled=True
        )

        # Buttons
        self.preview_button = widgets.Button(
            description='Preview Prompt',
            button_style='primary',
            layout=widgets.Layout(width='auto')
        )

        self.save_button = widgets.Button(
            description='Save Prompt',
            button_style='success',
            layout=widgets.Layout(width='auto')
        )

        self.copy_button = widgets.Button(
            description='Copy to Clipboard',
            button_style='info',
            layout=widgets.Layout(width='auto')
        )

        self.clear_button = widgets.Button(
            description='Clear All',
            button_style='warning',
            layout=widgets.Layout(width='auto')
        )

        # Output areas
        self.preview_output = widgets.Output()
        self.status_output = widgets.Output()

    def setup_layout(self):
        # Connect callbacks
        self.search_input.observe(self.on_search_change, names='value')
        self.example_list.observe(self.on_example_selected, names='value')
        self.tone_dropdown.observe(self.on_tone_change, names='value')
        self.preview_button.on_click(self.on_preview_clicked)
        self.save_button.on_click(self.on_save_clicked)
        self.copy_button.on_click(self.on_copy_clicked)
        self.clear_button.on_click(self.on_clear_clicked)

        # Display layout
        display(HTML("<h2>Prompt Builder</h2>"))

        # Examples section
        display(HTML("<h3>Browse Examples</h3>"))
        display(self.search_input)
        display(self.example_list)

        # Main input section
        display(HTML("<h3>Build Your Prompt</h3>"))

        # Display input fields with headers instead of labels
        display(HTML("<p><small>Task:</small></p>"))
        display(self.task_input)

        display(HTML("<p><small>Context:</small></p>"))
        display(self.context_input)

        display(HTML("<p><small>Constraints:</small></p>"))
        display(self.constraints_input)

        display(HTML("<p><small>Format:</small></p>"))
        display(self.format_input)

        display(HTML("<p><small>Tone:</small></p>"))
        display(self.tone_dropdown)
        display(self.custom_tone_input)

        # Buttons in a horizontal box
        button_box = widgets.HBox([
            self.preview_button,
            self.save_button,
            self.copy_button,
            self.clear_button
        ], layout=widgets.Layout(justify_content='space-between'))
        display(button_box)

        # Output areas
        display(self.preview_output)
        display(self.status_output)

    # Rest of the methods remain the same...
    def on_search_change(self, change):
        search_term = change['new'].lower()
        filtered_df = self.df[
            self.df['act'].str.lower().str.contains(search_term) |
            self.df['prompt'].str.lower().str.contains(search_term)
        ]
        self.example_list.options = [('Select an example...', -1)] + [
            (f"{row['act']}", i) for i, row in filtered_df.iterrows()
        ]

    def on_example_selected(self, change):
        if change['new'] != -1:
            example = self.df.iloc[change['new']]
            self.task_input.value = example['prompt']
            self.context_input.value = f"Acting as: {example['act']}"

    def on_tone_change(self, change):
        self.custom_tone_input.disabled = change['new'] != 'Custom'

    def generate_prompt(self):
        sections = []

        if self.task_input.value:
            sections.append(f"Task: {self.task_input.value}")

        if self.context_input.value:
            sections.append(f"Context: {self.context_input.value}")

        if self.constraints_input.value:
            sections.append(f"Constraints: {self.constraints_input.value}")

        if self.format_input.value:
            sections.append(f"Format: {self.format_input.value}")

        tone = self.custom_tone_input.value if self.tone_dropdown.value == 'Custom' else self.tone_dropdown.value
        sections.append(f"Tone: {tone}")

        return "\n\n".join(sections)

    def on_preview_clicked(self, b):
        with self.preview_output:
            clear_output()
            print("Generated Prompt:\n")
            print(self.generate_prompt())

    def on_save_clicked(self, b):
        prompt = self.generate_prompt()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"prompt_{timestamp}.txt"

        with open(filename, 'w') as f:
            f.write(prompt)

        with self.status_output:
            clear_output()
            print(f"Prompt saved to {filename}")

    def on_copy_clicked(self, b):
        prompt = self.generate_prompt()
        with self.status_output:
            clear_output()
            print("Prompt copied to clipboard!")

    def on_clear_clicked(self, b):
        self.task_input.value = ''
        self.context_input.value = ''
        self.constraints_input.value = ''
        self.format_input.value = ''
        self.tone_dropdown.value = 'Professional'
        self.custom_tone_input.value = ''
        self.search_input.value = ''
        self.example_list.value = -1

        with self.status_output:
            clear_output()
            print("All fields cleared")

        with self.preview_output:
            clear_output()

# Create and run the prompt builder
prompt_builder = PromptBuilder()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

prompts.csv:   0%|          | 0.00/84.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/170 [00:00<?, ? examples/s]

...

Textarea(value='', layout=Layout(height='100px', width='100%'), placeholder='Specify how you want the response…

Dropdown(layout=Layout(width='100%'), options=('Professional', 'Casual', 'Academic', 'Creative', 'Technical', …

Text(value='', disabled=True, layout=Layout(width='100%'), placeholder='Enter custom tone')

Output()

Output()